## About this notebook

**Objective**

Explore methodologies of financed emissions, using the automotive sector as an example.

Referenced methodology documents include [PCAF's 2022 Financed Emissions Standard](https://carbonaccountingfinancials.com/files/downloads/PCAF-Global-GHG-Standard.pdf), [Scotiabank's Automotive Sector Emissions Targeting](https://www.scotiabank.com/content/dam/scotiabank/corporate/Documents/EN-Emissions_Reduction_Target_Automotive_Sector_1-11-23.pdf), and [Scotiabank's 2023 Climate Report](https://www.scotiabank.com/content/dam/scotiabank/corporate/Documents/Scotiabank_2023_Climate_Report_Final.pdf).

**Data Overview**

To facilitate this exercise, a micro dataset was created using OEM’s publicly reported emissions and financial data from 2021.
Applied transformations include aligning reporting units for emissions, sales volumes, and debt + equity base.

*Note: The fiscal year end for which companies reported financials and emissions differed - thus the data collection periods across companies are not the same. While the values are still representative enough for this exercise, it points to one of many intricacies of accurately deriving financed emissions.*


---

**Data Dictionary**

| Column Name | Description |
|---|---|
| company | Company name |
| scope1 | Scope 1 emissions (tCO2e) |
| scope2 | Scope 2 emissions (tCO2e) |
| scope3 | Scope 3 emissions (tCO2e) |
| sales_vol | Vehicles sold (millions of units) |
| debt_eq | Debt + Equity base (millions USD) |

*Note: Data for Volvo is for the Volvo Cars business unit only*

## Package, Data, and User Defined Function Setup
You may run the cells in this code section all together to continue. You may also expand the code section to take a closer look and run cells individually.

In [1]:
import numpy as np
import pandas as pd
import random
import plotly.express as px

In [2]:
data = pd.read_csv('auto-client-data.csv', header = 0)
funds = 500

In [4]:
# Function to perform calculations based on allocation selection

def calculations(data, selection, funds):
  # Subsetting to the selected companies
  df = data[data['company'].isin(selection.keys())].reset_index(drop=True)

  # Create column for authorized amount
  df['authorized'] = df['company'].map(selection)

  # Create column for amount outstanding (random value upwards of 30%)

  df['outstanding'] = df['authorized'].apply(lambda x: x * random.choice([0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])).astype(int)

  # Calculate per client total emissions in megatonnes
  df['total_em'] = round((df['scope1'] + df['scope2'] + df['scope3']) / 1_000_000, 4)

  # Calculate per client emissions intensity proxy
  df['prod_intensity'] = round(df['total_em'] / df['sales_vol'], 2)

  # Calculate attribution ratio
  df['attr_ratio'] = df['outstanding'] / df['debt_eq']

  # Calculate portfolio percentage
  df['perc'] = df['authorized'] / funds

  # Calculate absolute financed emissions in megatonnes and tonnes for printing
  df['abs_em'] = df['attr_ratio'] * df['total_em']
  df['abs_em_tonnes'] = (df['attr_ratio'] * df['total_em'] * 1_000_000).astype(int)

  return df

In [5]:
def analyze_portfolio(results_df):

  # Print absolute financed emissions values
  for index, row in results_df.iterrows():
      print(f"The ${row['authorized']} million loaned to {row['company']} generated {row['abs_em_tonnes']:,d} tCO2e")

  # Calculate and print overall physical emissions intensity
  portfolio_intensity = np.average(results_df['prod_intensity'], weights=results_df['perc'])
  print(f"\nThe physical emissions intensity of this loan portfolio is {portfolio_intensity:.2f}")

  # Create and display a donut chart for loan portfolio allocations
  fig_donut = px.pie(
      results_df,
      names='company',
      values='authorized',
      title='Loan Portfolio Allocations Across Companies',
      hole=0.4
  )
  fig_donut.show()

  # Create and display a horizontal bar chart for absolute financed emissions
  fig_em = px.bar(
      results_df,
      x='abs_em_tonnes',
      y='company',
      title='Absolute Financed Emissions by Company',
      labels={'abs_em_tonnes': 'Absolute Financed Emissions (tCO2e)', 'company': 'Company'},
      orientation='h'
  )
  fig_em.show()

  # Create and display a horizontal bar chart for physical emissions intensity
  fig_pet = px.bar(
      results_df,
      x='prod_intensity',
      y='company',
      title='Physical Emissions Intensity by Company',
      labels={'prod_intensity': 'Physical Emissions Intensity (Emissions relative to vehicles sold)', 'company': 'Company'},
      orientation='h'
  )
  fig_pet.show()

In [6]:
def explain_abs_em(results_df):
  explanation = f"""
  You loaned ${results_df.loc[0, 'authorized']} million to {results_df.loc[0, 'company']}.
  Of that ${results_df.loc[0, 'authorized']} million, {results_df.loc[0, 'company']} used ${results_df.loc[0, 'outstanding']} million.

  This amount that {results_df.loc[0, 'company']} accessed is used to calculate your attribution ratio,  representing your contribution to {results_df.loc[0, 'company']}'s activities for the year.
  This ratio is calculated as the ${results_df.loc[0, 'outstanding']} million outstanding divided by  {results_df.loc[0, 'company']}'s debt and equity that year (${results_df.loc[0, 'debt_eq']:,d} million),
  which yields a value of approximately {round(results_df.loc[0, 'attr_ratio'] * 100, 5)}%.

  This ratio is then multiplied by {results_df.loc[0, 'company']}'s total emissions (across scopes 1, 2, and 3),  which came to {results_df.loc[0, 'total_em']} MtCO2e.
  Expressed in tCO2e (converted from MtCO2e), this gives you your financed emissions value of {results_df.loc[0, 'abs_em_tonnes']:,d} tCO2e.
  """

  print(explanation)

In [7]:
def explain_pet(results_df, funds):

  portfolio_intensity = np.average(results_df['prod_intensity'], weights=results_df['perc'])

  explanation = """
  Recall that for a single company, their emissions intensity is a normalized view of their emissions relative to their
  economic productivity.
  In the absence of consistent information about production and average lifetime vehicle kilometers across produced vehicle
  classes, we used the number of vehicles sold as a rough proxy.

  Thus, for each company you loaned to, we calculated their emissions intensity as
  their total emissions divided by vehicles sold for the year.
  """

  # Add emissions intensity explanation for each company
  for index, row in results_df.iterrows():
      explanation += f"""
      For {row['company']}, this would be {row['total_em']} MtCO2e divided by {row['sales_vol']} million vehicles sold
      --> {row['prod_intensity']} emissions intensity.
      """

  # Add portfolio proportion explanation for each company
  explanation += "\nWe also computed the proportion of your total funds loaned out to each company,\nrepresenting its proportion of your auto sector financing.\n"
  for index, row in results_df.iterrows():
      explanation += f"""
      For {row['company']}, that would be ${row['authorized']} million divided by ${funds} million
      --> {round(row['perc'] * 100, 2)}%.
      """

  # Add portfolio intensity explanation
  explanation += f"""
  Taking a weighted average of the proportion per company and the emissions intensity per company,
  we arrive at a physical emissions intensity of {portfolio_intensity}.
  """

  print(explanation)

## The Scenario
It is 2020 and you are tasked with allocating loan funding to select automakers on behalf of your institution. You have $500 million which must be allocated to four different firms.
After 1 year, you will evaluate your resulting climate metrics. Namely, your **absolute financed emissions** as well as the **phyisical emissions intensity** for your auto sector loan portfolio.

**Understanding Absolute Emissions:**
This measures the share of the companies' emissions financed by your loans.

**Understanding Physical Emissions Intensity:**
For a given company, physical emissions intensity provides a view of its emissions relative to its economic activity. The resulting metrics allows better comparability across companies or portfolios.

A sample selection is presented below for walking through the exercise. If you'd like, you may edit the companies and loan authorization amounts in the dictionary. You can rerun the code cells after this to see how your results change.

Valid companies to select include Tesla, Stellantis, Subaru, Kia, Nissan, Hyundai, Honda, BMW, and Volvo.

For a smoother experience with loan authorization amounts, it is recommended to allocate the full $500 million across 4 unique companies and use step values of \$50 million

In [8]:
# The example allocation remains here for your reference  {'Tesla': 150, 'Honda': 50, 'Volvo': 200, 'Kia': 100}
allocation = {'Tesla': 150, 'Honda': 50, 'Volvo': 200, 'Kia': 100}

## The Results
Run the cells in this code section to calculate and view your results.

In [9]:
results_df = calculations(data = data, selection = allocation, funds = funds)

In [10]:
# Scroll up within the cell output to start from the top with the text summary  (rather than the bar graphs)
analyze_portfolio(results_df)

The $150 million loaned to Tesla generated 3,068 tCO2e
The $100 million loaned to Kia generated 90,974 tCO2e
The $50 million loaned to Honda generated 27,307 tCO2e
The $200 million loaned to Volvo generated 47,580 tCO2e

The physical emissions intensity of this loan portfolio is 22.82


## Results Walkthrough

### Calculating Absolute Emissions
Let's walk through an example to understand how the absolute financed emissions were calculated

In [11]:
explain_abs_em(results_df)


  You loaned $150 million to Tesla.
  Of that $150 million, Tesla used $75 million.

  This amount that Tesla accessed is used to calculate your attribution ratio,  representing your contribution to Tesla's activities for the year.
  This ratio is calculated as the $75 million outstanding divided by  Tesla's debt and equity that year ($62,131 million),
  which yields a value of approximately 0.12071%.

  This ratio is then multiplied by Tesla's total emissions (across scopes 1, 2, and 3),  which came to 2.542 MtCO2e.
  Expressed in tCO2e (converted from MtCO2e), this gives you your financed emissions value of 3,068 tCO2e.
  


### Calculating Physical Emissions Intensity
Now let's walk through how this value was derived.

Recall that for a single company, their emissions intensity is a normalized view of their emissions relative to their economic productivity. In the asbcence of consistent information about production and average lifetime vehicle kilometres across produced vehicle classes, we used number of vehicles sold as a rough proxy.

Thus, for each company you loaned to, we calculated their emissions intensity as their total emissions divided by vehicles sold for the year.

In [12]:
explain_pet(results_df, funds)


  Recall that for a single company, their emissions intensity is a normalized view of their emissions relative to their
  economic productivity.
  In the absence of consistent information about production and average lifetime vehicle kilometers across produced vehicle
  classes, we used the number of vehicles sold as a rough proxy.

  Thus, for each company you loaned to, we calculated their emissions intensity as
  their total emissions divided by vehicles sold for the year.
  
      For Tesla, this would be 2.542 MtCO2e divided by 0.94 million vehicles sold
      --> 2.7 emissions intensity.
      
      For Kia, this would be 68.7024 MtCO2e divided by 2.66 million vehicles sold
      --> 25.83 emissions intensity.
      
      For Honda, this would be 254.48 MtCO2e divided by 4.55 million vehicles sold
      --> 55.93 emissions intensity.
      
      For Volvo, this would be 19.683 MtCO2e divided by 0.7 million vehicles sold
      --> 28.12 emissions intensity.
      
We also comp

## End Notes

Through this simplified example, you've learned a little bit about how institutions measure their financed emissions.

🚧 **Coming Soon** 🚧

Interactive Streamlit App with a more user friendly UI